# loop

在编译原理课程项目中，进行自然循环识别的代码。

## 需要的库

- 正则表达式 re
- 序列化 json

In [2]:
import re
import json
import copy

## 读入json

In [3]:
filename = "../examples/vm_src02_blk.json"
vm_blk = {}
with open(filename, "r") as fp:
    vm_blk = json.load(fp)
blocks = vm_blk["blocks"]

cond_f = {"==": lambda x, y: x == y, "!=": lambda x, y: x != y, ">=": lambda x, y: x >=
          y, "<=": lambda x, y: x <= y, ">": lambda x, y: x > y, "<": lambda x, y: x < y}
arith_f = {"+": lambda x, y: x+y, "-": lambda x, y: x-y, "*": lambda x,
           y: x*y, "/": lambda x, y: x//y, "%": lambda x, y: x % y}
eserved_word = {"HALT", "=", "+", "-", "*", "/", "%", "?",
                ":", "!:", ">", "<", ">=", "<=", "==", "!=", "[", "]"}
reg = re.compile('^[0-9]+$')

In [4]:
blocks

{'0': {'line_num': [0, 3],
  'next': [1, None],
  'code': ['I = M - 1', 'J = N', 'T_1 = N', 'V = A [ T_1 ]'],
  'defd': ['T_1', 'J', 'I', 'V'],
  'used': ['A', 'N', 'M'],
  'in': ['M', 'A', 'N'],
  'out': ['A', 'I', 'V', 'J', 'N']},
 '1': {'line_num': [4, 7],
  'next': [2, 1],
  'code': ['I = I + 1', 'T_2 = I', 'T_3 = A [ T_2 ]', '? V > T_3 : 4'],
  'defd': ['T_3', 'T_2'],
  'used': ['V', 'A', 'I'],
  'in': ['A', 'I', 'V', 'J', 'N'],
  'out': ['A', 'I', 'V', 'J', 'N']},
 '2': {'line_num': [8, 11],
  'next': [3, 2],
  'code': ['J = J - 1', 'T_4 = J', 'T_5 = A [ T_4 ]', '? T_5 > V : 8'],
  'defd': ['T_4', 'T_5'],
  'used': ['V', 'A', 'J'],
  'in': ['A', 'I', 'V', 'J', 'N'],
  'out': ['A', 'I', 'V', 'J', 'N']},
 '3': {'line_num': [12, 12],
  'next': [4, 5],
  'code': ['? J <= I : 22'],
  'defd': [],
  'used': ['J', 'I'],
  'in': ['V', 'A', 'J', 'I', 'N'],
  'out': ['A', 'I', 'V', 'J', 'N']},
 '4': {'line_num': [13, 21],
  'next': [1, None],
  'code': ['T_6 = I',
   'X = A [ T_6 ]',
   'T_

# 给基本块加入前驱节点属性

In [5]:
for n,b in blocks.items():
    blocks[n]["pre"] = set()

for n,b in blocks.items():
    for i in b["next"]:
        if i!=None:
            blocks[str(i)]["pre"] = blocks[str(i)]["pre"] | {n}
# blocks['0']["pre"] = {'E'}
for n,b in blocks.items():
    # blocks[n]["pre"] = list(blocks[n]["pre"])
    print(n,b["pre"])

0 set()
1 {'0', '1', '4'}
2 {'1', '2'}
3 {'2'}
4 {'3'}
5 {'3'}


# 构建支配关系

In [6]:
for n,b in blocks.items():
    blocks[n]["dom"] = set([str(i) for i in range(vm_blk["summary"]["total_blocks"])])
blocks[str(0)]["dom"] = {'0'}

# for n,b in blocks.items():
#     print(n,b["dom"])

flag = True
i = 0
while flag:
    flag = False
    i+=1
    # print("#",i)
    for n,b in blocks.items():
        if n!='0':
            domset_ori = copy.deepcopy(blocks[n]["dom"])
            domset = copy.deepcopy(blocks[n]["dom"])
            preset = set([str(i) for i in range(vm_blk["summary"]["total_blocks"])])
            for j in blocks[n]["pre"]:
                preset = preset & blocks[j]["dom"]
            domset = {n} | preset
            if (domset!=domset_ori):
                flag = True
            blocks[n]["dom"] = domset
    # for n,b in blocks.items():
    #     print(n,b["dom"])
for n,b in blocks.items():
    if len(b["pre"])==0 and n!='0':
        blocks[n]["dom"] = set()

for n,b in blocks.items():
    print(n,b["dom"])

0 {'0'}
1 {'0', '1'}
2 {'0', '1', '2'}
3 {'0', '3', '2', '1'}
4 {'3', '0', '4', '2', '1'}
5 {'5', '3', '0', '1', '2'}


# 寻找回边

In [7]:
stack = []
stack.append(1)
stack.append(2)
stack.append(3)
print(stack)
stack.pop()
print(stack)


[1, 2, 3]
[1, 2]


In [8]:
stack = ['0']
visited = {str(i):False for i in range(vm_blk["summary"]["total_blocks"])}
dfn = {str(i):vm_blk["summary"]["total_blocks"]*7 for i in range(vm_blk["summary"]["total_blocks"])}
low = {str(i):vm_blk["summary"]["total_blocks"]*7 for i in range(vm_blk["summary"]["total_blocks"])}
parent = {str(i):None for i in range(vm_blk["summary"]["total_blocks"])}
cnt = 0
back_edge = []
while len(stack)>0:
    tmp = stack[-1]
    visited[tmp] = True
    stack.pop()
    # print(tmp)
    for i in blocks[tmp]["next"]:
        if (i!=None):
            if not visited[str(i)]:
                stack.append(str(i))
                dfn[str(i)] = cnt
                cnt+=1
                parent[str(i)] = tmp
            else:
                # print(tmp,dfn[tmp],"->",str(i),dfn[str(i)])
                back_edge.append((tmp,str(i)))
print(dfn)
print(parent)
print(back_edge)

{'0': 42, '1': 0, '2': 1, '3': 2, '4': 3, '5': 4}
{'0': None, '1': '0', '2': '1', '3': '2', '4': '3', '5': '3'}
[('1', '1'), ('2', '2'), ('4', '1')]


In [9]:
back_edge = []
for n,b in blocks.items():
    for i in b["dom"]:
        if int(i) in b["next"]:
            back_edge.append((n,i))
print(back_edge)

[('1', '1'), ('2', '2'), ('4', '1')]


# 寻找一条回边对应的自然循环

In [10]:
loops = []
for le in back_edge:
    stack = [le[0]]
    loop = {le[0],le[1]}
    while len(stack)>0:
        m=stack[-1]
        stack.pop()
        for p in blocks[m]["pre"]:
            if p not in loop:
                loop = loop | {p}
                stack.append(p)
    loop = [i for i in list(loop) if len(blocks[i]["pre"])>0]
    # print(le,sorted(loop))
    loops.append(loop)
loops = sorted(loops,key = lambda x:len(x))
print(loops)

[['3', '4', '1', '2'], ['3', '4', '1', '2'], ['4', '2', '3', '1']]
